# BRCA1

This notebook shows how to develop a classifier with embedded tests in Jupyter.

In [ ]:
# !uv pip install -e ../../python

In [1]:
from bioscript.classifier import DiploidResult, GenotypeClassifier, GenotypeEnum
from bioscript.types import Alleles, VariantCall

In [2]:
import pandas as pd

In [3]:
def filter_snvs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return only rows where clnvc == 'single_nucleotide_variant' (case-insensitive).
    """
    mask = df["clnvc"].str.lower() == "single_nucleotide_variant"
    return df[mask].reset_index(drop=True)

In [4]:
def generate_variant_calls(df: pd.DataFrame) -> list[VariantCall]:
    """
    Generate VariantCall objects from ClinVar DataFrame.
    Now handles all variant types including duplications, deletions, and indels.
    """
    vcs: list[VariantCall] = []
    for _, row in df.iterrows():
        rsid_value: str | None = None
        if not pd.isna(row["rsid"]):
            candidate = str(row["rsid"]).strip()
            invalid_tokens = {".", "<NA>", "NA", "N/A"}
            if candidate and candidate.upper() not in invalid_tokens:
                rsid_value = candidate

        ref = str(row["ref"]).strip().upper()
        alt = str(row["alt"]).strip().upper()
        chrom = None
        if not pd.isna(row["chromosome"]):
            chrom_candidate = str(row["chromosome"]).strip()
            if chrom_candidate:
                chrom = chrom_candidate
        pos = None
        if not pd.isna(row["position"]):
            pos = int(row["position"])

        # Handle multi-character REF and ALT for indels/duplications
        # For now, just use the first character for matching purposes
        # The full sequences are stored in ref_label and alt_label
        try:
            # For single nucleotides, use from_letter
            if len(ref) == 1:
                ref_alleles = Alleles.from_letter(ref)
            else:
                # For multi-character ref (indels), use first base
                ref_alleles = Alleles.from_letter(ref[0])
                
            if len(alt) == 1:
                alt_alleles = Alleles.from_letter(alt)
            else:
                # For multi-character alt (indels), use first base
                alt_alleles = Alleles.from_letter(alt[0])
        except:
            # If we can't parse, skip this variant
            print(f"Warning: Skipping variant {rsid_value} with ref={ref} alt={alt}")
            continue

        vc = VariantCall(
            rsid=rsid_value,
            ref=ref_alleles,
            alt=alt_alleles,
            chromosome=chrom,
            position=pos,
        )
        # Store original full sequences for debugging
        vc.ref_label = ref
        vc.alt_label = alt

        vcs.append(vc)

    return vcs

In [5]:
def get_vcs():
    # Path to your TSV file
    tsv_path = "brca1_clinvar.tsv"
    
    # Load the TSV file
    df = pd.read_csv(
        tsv_path,
        sep="\t",
        dtype={
            "rsid": "string",
            "gene": "string",
            "chromosome": "string",
            "position": "Int64",
            "ref": "string",
            "alt": "string",
            "clnrevstat": "string",
            "clnsig": "string",
            "clnvc": "string",
        }
    )

    print(f"Using all {len(df)} rows")
    
    # Count variant types for debugging
    variant_types = df["clnvc"].value_counts()
    print("\nVariant types in dataset:")
    for vtype, count in variant_types.items():
        print(f"  {vtype}: {count}")

    df_snvs = filter_snvs(df)
    
    print(f"\nGenerating variant calls for all {len(df_snvs)} variants")
    vcs = generate_variant_calls(df_snvs)  # Using full df, not filtered
    print(f"Generated {len(vcs)} variant calls")
    return vcs

In [6]:
class BRCA1Classifier(GenotypeClassifier):
    def classify(self, matches) -> DiploidResult:
        if not matches.all_matches:
            print("No variant matches were found.", flush=True)

        buckets = {
            "REFERENCE_CALL": [],
            "VARIANT_CALL": [],
            "NO_CALL": [],
        }

        def format_genotype(snp):
            from bioscript.types import Nucleotide

            symbols = []
            for allele in snp:
                if isinstance(allele, Nucleotide) and allele == Nucleotide.MISSING:
                    symbols.append('-')
                else:
                    symbols.append(allele.value)
            return ''.join(symbols)

        for match in matches.all_matches:
            call = match.variant_call
            chrom = call.chromosome or "?"
            pos = call.position if call.position is not None else "?"
            if call.rsid:
                if hasattr(call.rsid, "aliases"):
                    rsid_repr = ", ".join(sorted(call.rsid.aliases))
                else:
                    rsid_repr = str(call.rsid)
            else:
                rsid_repr = f"chr{chrom}:{pos}"

            ref = getattr(
                call,
                "ref_label",
                "/".join('-' if n.value == '.' else n.value for n in sorted(call.ref, key=lambda x: x.value)),
            )
            alt = getattr(
                call,
                "alt_label",
                "/".join('-' if n.value == '.' else n.value for n in sorted(call.alt, key=lambda x: x.value)),
            )
            genotype = format_genotype(match.snp)
            line = (
                f"{match.match_type.name:<14} {rsid_repr} chr{chrom}:{pos} "
                f"ref={ref} alt={alt} genotype={genotype}"
            )
            buckets.setdefault(match.match_type.name, []).append(line)

        def emit(header: str, lines: list[str]):
            if not lines:
                return
            print(header, flush=True)
            for idx, line in enumerate(lines, 1):
                print(f"  {idx:>3} {line}", flush=True)

        emit("Reference matches", buckets["REFERENCE_CALL"])
        emit("Variant matches", buckets["VARIANT_CALL"])
        emit("Diagnostics (no call)", buckets["NO_CALL"])

        return DiploidResult("DEBUG", "DEBUG")


In [7]:
__bioscript__ = {
    "variant_calls": get_vcs(),
    "classifier": BRCA1Classifier(),
    "name": "BRCA1",
}

Using all 2177 rows

Variant types in dataset:
  Deletion: 1034
  single_nucleotide_variant: 523
  Duplication: 344
  Insertion: 203
  Indel: 73

Generating variant calls for all 523 variants
Generated 523 variant calls


## Tests

Write tests using the test_* function convention:

In [8]:
# from bioscript import VariantFixture
# from bioscript.types import MatchList

# fixture = VariantFixture(
#     [
#         {"rsid": "rs73885319", "chromosome": "22", "position": 36265860},
#         {"rsid": "rs60910145", "chromosome": "22", "position": 36265988},
#         {"rsid": "rs71785313", "chromosome": "22", "position": 36266000},
#     ],
#     assembly="GRCh38",
# )

In [9]:
# def test_g0_homozygous():
#     variants = fixture(["AA", "TT", "II"])
#     matches = MatchList([rs73885319, rs60910145, rs71785313]).match_rows(variants)
#     classifier = APOL1Classifier()
#     result = classifier(matches)
#     assert result == "G0/G0"

In [10]:
# def test_g1_homozygous():
#     variants = fixture(["GG", "CC", "II"])
#     matches = MatchList([rs73885319, rs60910145, rs71785313]).match_rows(variants)
#     classifier = APOL1Classifier()
#     result = classifier(matches)
#     assert result == "G1/G1"

## Run Tests in Jupyter

You can run tests directly in the notebook:

In [11]:
# # Run tests
# test_g0_homozygous()
# test_g1_homozygous()
# print("✓ All tests passed!")

## Export to Python Module

Export this notebook to a Python file:

```bash
bioscript export apol1_dev.ipynb -o classify_apol1_exported.py
```

Or in Python:

```python
from bioscript import export_from_notebook
export_from_notebook("apol1_dev.ipynb", "classify_apol1_exported.py")
```

In [12]:
from bioscript import export_from_notebook
export_from_notebook("brca1_dev.ipynb", "brca1_classifier.py")

PosixPath('brca1_classifier.py')

In [13]:
# !bioscript test brca1_classifier.py

In [14]:
# Run the classifier - should be fast now!
!bioscript classify brca1_classifier.py --participant_id="X" --file="carika.txt" --out=tsv

Using all 2177 rows

Variant types in dataset:
  Deletion: 1034
  single_nucleotide_variant: 523
  Duplication: 344
  Insertion: 203
  Indel: 73

Generating variant calls for all 523 variants
Generated 523 variant calls
Reference matches
    1 REFERENCE_CALL rs886040303 chr17:43045728 ref=G alt=A genotype=GG
    2 REFERENCE_CALL rs397509295 chr17:43045729 ref=G alt=T genotype=GG
    3 REFERENCE_CALL rs80356977 chr17:43045735 ref=G alt=T genotype=GG
    4 REFERENCE_CALL rs80357107 chr17:43045757 ref=A alt=T genotype=AA
    5 REFERENCE_CALL rs80356914 chr17:43045759 ref=C alt=T genotype=CC
    6 REFERENCE_CALL rs80356959 chr17:43045761 ref=A alt=G genotype=AA
    7 REFERENCE_CALL rs80356942 chr17:43045764 ref=C alt=A genotype=CC
    8 REFERENCE_CALL rs80358048 chr17:43045803 ref=C alt=T genotype=CC
    9 REFERENCE_CALL rs80356868 chr17:43047661 ref=C alt=A genotype=CC
   10 REFERENCE_CALL rs397509284 chr17:43047665 ref=C alt=T genotype=CC
   11 REFERENCE_CALL rs80356962 chr17:43047666 re